# Introduction

This R Markdown document shows the usage of the package `cino1`. 

In [ ]:
# load package
library(cinof1)
library(doParallel)

## Data

In this package, a sample data frame is included. It contains data for 300 patients within an n of 1 study. The data has the following structure:

* _patient_id_: Unique patient identifier
* _date_: Date of data points
* _day_: Day in study
* _Block_: identifies treatment block
* _Activity_: Dummy variable for steps per day
* _treatment_: Dummy variable for 2 treatments as factors
* _Uncertain_Low_Back_Pain_: Dummy variable for Uncertain loq back pain on scale 1-15

In [ ]:
load("data/simpatdat.rda")
# Summarize Data
summary(simpatdat)

## Basic Functions

Basic functions for analyse N-of-1 studys are for example wilcox test or comparative plots. These two functions are provided in this package.

### Comparative Plot

To get a first idea about the data and the difference between treatment 1 and treatment 2, a comparative plot could be used. It shows the outcome on the y-Axis against the different treatments on the x-Axis,

In [ ]:
# Define outcome and exposure column
outcome <- "Uncertain_Low_Back_Pain"
exposure <- "treatment"
# Plot outcome among different exposures
comparative.plot(simpatdat, exposure = exposure, outcome = outcome)

## Wilcox Test

To validate, that there is a difference in both treatments, the Wilcox test could be used. It calculates the p-value for the null hypothesis, that there location shift is equal to zero.

In [ ]:
# Define outcome and exposure column
outcome <- "Uncertain_Low_Back_Pain"
exposure <- "treatment"
# Perform Wilcox test among different exposures
wilcox.nofone(simpatdat, exposure = exposure, outcome = outcome)

## Adjust Wash In and Wash Out


In [ ]:
outcome <- "Uncertain_Low_Back_Pain"
exposure <- "treatment"
variables <- c("Activity")
id <- "patient_id"
time_col <- "day"

result <- estimate.gamma.tau(data = simpatdat, outcome = outcome, exposure = exposure, variables = variables, bound = 2, symmetric = TRUE, id=id, time_col = time_col)

result

fit.adj.lm(data = simpatdat, outcome = outcome, exposure = exposure, variables = variables, effects = result$best, id = id, time_col = time_col)

## Bayesian

Bayesian Networks are used to calculated the probability of outcome variables adjusted for confounders. For that, a dag is required, which identifies the relations between the variables.
In this implementation, also lags are included and could be specified in the dag by adding `.lag=` to the variable name.

###  Preprocess Data

In [ ]:
# specify column names
id <- "patient_id"
time_col <- "day"

# Load data
load("data/simpatdag.rda")
load("data/simpatdat.rda")

# Dag preprocessing
bn.dag <- bn.prep.dag(simpatdag)

# Data Preprocessing (Factorization)
simpatdat$Uncertain_Low_Back_Pain <- as.factor(simpatdat$Uncertain_Low_Back_Pain)
simpatdat$Activity <- cut(simpatdat$Activity, 3, labels=c("low Activity", "middle Activity", "high Activity"))
bn.data <- bn.prep.data(bn.dag, simpatdat, id, time_col)
bn.data <- na.omit(bn.data)

### Fit and Plot Results

In [ ]:
fitted.bn <- bn.fit.dag(bn.data, bn.dag, method="bayes")

library(bnlearn)
bn.fit.barchart(fitted.bn$Uncertain_Low_Back_Pain)

## G-Estimation

G-Estimation is used to adjust the analysis for causal inferences. For that, three different methods are implemented

### Load Data

In [ ]:
load("data/simpatdat.rda")

### Fit G-Estimation by Iteration
It iterates over several values for $\psi$ and returns a data frame with $\psi$ and corresponding $\alpha$

In [ ]:
outcome <- "Uncertain_Low_Back_Pain"
exposure <- "treatment"
confounder <- c("Activity")
id <- "patient_id"
df <- nofgest(simpatdat, outcome, exposure, confounder, id, method="iterate", steps=100, upper_bound_psi = 10, lower_bound_psi = -10)

This function is useful to plot a curve for $\alpha$ and $\psi$.

In [ ]:
plot( df$PSI ,df$Beta, type="l", main=expression(paste("Plot of ", alpha, "( ", psi,")")),
        xlab=expression(psi),
        ylab=expression(alpha))
# Add a second line
lines( c(-100,100),c(0,0), type = "l", col = "red")

### Fit G-Estimation by Recursive Mean

This function approximate $\psi$ by an interval search.

In [ ]:
outcome <- "Uncertain_Low_Back_Pain"
exposure <- "treatment"
confounder <- c("Activity")
id <- "patient_id"
nofgest(simpatdat, outcome, exposure, confounder, id, method="rec_mean", max_number_it = 10, verbose=FALSE)

### Fit G-Estimation by Recursive Improved

This function approximate $\psi$ by an optimized interval search.

In [ ]:
outcome <- "Uncertain_Low_Back_Pain"
exposure <- "treatment"
confounder <- c("Activity")
id <- "patient_id"
nofgest(simpatdat, outcome, exposure, confounder, id, method="rec", max_number_it = 10, verbose=FALSE)